In [1]:
import pandas as pd
import pyodbc
import numpy as np
import os

In [2]:
path = r'T:\Tableau\tableauTitleVI\Datasources'

In [3]:
data = pd.read_csv(path + '\\TitleVIchangeovertime.csv')   
df = pd.DataFrame(data)

print(df)

     Year      Factor                   Universe  FactorTotal  MPOTotal  \
0    2007    Disabled  Non-Institutionalized Pop      33553.0  219933.0   
1    2007     Elderly                 Population      30468.0  237262.0   
2    2007    Minority                 Population      38828.0  237262.0   
3    2007     Poverty                 Households      16940.0   98311.0   
4    2007     Renters                 Households      42082.0   98311.0   
..    ...         ...                        ...          ...       ...   
100  2021    Minority                 Population      63125.0  268190.0   
101  2021     Poverty                 Households      15907.0  113975.0   
102  2021     Renters                 Households      51861.0  113975.0   
103  2021  Unemployed        Workforce (Pop 16+)      10128.0  144512.0   
104  2021   Zero Cars                 Households       9208.0  113975.0   

       MPOavg  
0    0.152560  
1    0.128415  
2    0.163650  
3    0.172310  
4    0.428050  
.. 

In [4]:
data.columns

Index(['Year', 'Factor', 'Universe', 'FactorTotal', 'MPOTotal', 'MPOavg'], dtype='object')